In [1]:
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
import unicodedata
import re

import sys
from tqdm.notebook import tqdm
from IPython.display import display, HTML, clear_output
import random

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
claim_train = pd.read_csv('../data/train_claim_embeddings.csv', low_memory=False)
claim_test = pd.read_csv('../data/test_claim_embeddings.csv', low_memory=False)
claim_val = pd.read_csv('../data/validation_claim_embeddings.csv', low_memory=False)

In [3]:
main_train = pd.read_csv('../data/train_main_text_embeddings.csv', low_memory=False)
main_test = pd.read_csv('../data/test_main_text_embeddings.csv', low_memory=False)
main_val = pd.read_csv('../data/validation_main_text_embeddings.csv', low_memory=False)

In [4]:
merged_train = main_train.merge(claim_train, how='inner', on='claim_id')
merged_test = main_test.merge(claim_test, how='inner', on='claim_id')
merged_val = main_val.merge(claim_val, how='inner', on='claim_id')

In [5]:
column_x = []
column_y = []

for i in range(768):
    column_x.append(str(i)+'_x')
    column_y.append(str(i)+'_y')

In [6]:
def calculate_cosine_similarity(row):
    return cosine_similarity(row[column_x].values.reshape(1, -1), row[column_y].values.reshape(1, -1)).reshape(-1)[0]

In [7]:
merged_train['cosine_similarity'] = merged_train.apply(lambda row: calculate_cosine_similarity(row), axis=1)
merged_test['cosine_similarity'] = merged_test.apply(lambda row: calculate_cosine_similarity(row), axis=1)
merged_val['cosine_similarity'] = merged_val.apply(lambda row: calculate_cosine_similarity(row), axis=1)

In [8]:
def generate_pairs(df):
    return_map = dict()
    for claim_id in tqdm(df.claim_id.values):
        return_map[claim_id] = df.loc[df['claim_id'] == claim_id].sort_values(by='cosine_similarity', axis=0, ascending=False).head(5)['sent_id'].values
    return return_map

In [9]:
train_map = generate_pairs(merged_train)
test_map = generate_pairs(merged_test)
val_map = generate_pairs(merged_val)
clear_output()

In [10]:
len(train_map.keys()), len(test_map.keys()), len(val_map.keys())

(9805, 1235, 1215)

In [11]:
claim_train_df = pd.read_csv('../data/claim_train_df.csv', low_memory=False)
claim_test_df = pd.read_csv('../data/claim_test_df.csv', low_memory=False)
claim_val_df = pd.read_csv('../data/claim_val_df.csv', low_memory=False)

In [12]:
main_train_df = pd.read_csv('../data/main_text_train_df.csv', low_memory=False)
main_test_df = pd.read_csv('../data/main_text_test_df.csv', low_memory=False)
main_val_df = pd.read_csv('../data/main_text_val_df.csv', low_memory=False)

In [13]:
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

def remove_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_special_characters(text): 
    return re.sub(r'[^a-zA-z0-9.,!?/:;\"\'\s]', '', text)

def remove_extra_whitespace_tabs(text):
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

def preprocess(text):
    processed_text = remove_html_tags(text)
    processed_text = remove_accented_chars(processed_text)
    processed_text = remove_special_characters(processed_text)
    processed_text = remove_extra_whitespace_tabs(processed_text)
    return processed_text

def generate_pair_dataframes(claim_df, main_text_df, pair_map):
    return_df = pd.DataFrame(columns=['claim', 'main_text', 'label'])
    for key in tqdm(pair_map.keys()):
        claim = claim_df[claim_df['claim_id'] == key]['sentence'].values[0]
        label = claim_df[claim_df['claim_id'] == key]['label'].values[0]
        main_text = preprocess(' '.join(sentence for sentence in main_text_df[(main_text_df['claim_id'] == key) & (main_text_df['sent_id'].isin(pair_map[key]))]['sentence'].values))
        return_df.loc[len(return_df.index)] = [claim, main_text, label]
    return return_df

In [14]:
train = generate_pair_dataframes(claim_train_df, main_train_df, train_map)
test = generate_pair_dataframes(claim_test_df, main_test_df, test_map)
val = generate_pair_dataframes(claim_val_df, main_val_df, val_map)
clear_output()

In [15]:
def show_random_elements(df, num_examples=10):
    assert num_examples <= df.shape[0], "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, df.shape[0]-1)
        while pick in picks:
            pick = random.randint(0, df.shape[0]-1)
        picks.append(pick)
    
    display(HTML(df.iloc[picks].to_html()))

In [16]:
show_random_elements(train)

,claim,main_text,label
7406,"Mediterranean diet cuts heart, diabetes risk factors","The story described the health benefits associated with consumption of a Mediterranean diet in the broadest of terms. The story included data from the American Heart Association about the numbers /percentage of people who met the criteria for metabolic syndrome, and its prevalence in adolescents. The study was about risk factors for risk factors. Metabolic syndrome is not a disease. Other studies have compared the Mediterranean diet with a lower carb diet and others, and looked at intermediate endpoints that go along with the metabolic syndrome.",1
15,The train carrying GOP members of Congress to a West Virginia retreat crashed due to a Deep State plot.,"On 31 January 2018, an Amtrak train carrying Republican members of Congress to a retreat struck a garbage truck that was on the tracks at 11:20 a.m. in Crozet, Virginia, killing one person. Less than twelve hours after President Trump was caught on a hot mic telling a colleague that the memo would 100 be released, the Deep State killing machine whirred into life and attempted to cause mass carnage on the Amtrak rails. They claimed that U.S. spies and secret power brokers feared the address would rally the country behind Trump. According to witnesses, including lawmakers who were on board the train headed to a retreat in West Virginia, the train hit a garbage truck that was on the tracks. a massive Deep State conspiracy to harm Republican legislators.",0
2332,Viral image Says the coronavirus isn’t new because “Lysol has it listed as one of the viruses that it kills.”,"""Many social media users are sharing posts that suggest health officials are lying when they claim ignorance about the recent coronavirus outbreak, such as this one from Jan. 31. """"Dont fall for the B.S. they claim the Coronavirus is new... But regular over the counter Lysol has it listed as one of the viruses that it kills!"" Weve already debunked several hoaxes and conspiracies about the coronavirus, including one thats similar to the claim in this Facebook post: that Clorox bottles prove the coronavirus was developed before the recent outbreak in China. It says that specific Lysol products, including its disinfectant spray, """"have demonstrated effectiveness against viruses similar to 2019 Novel coronavirus 2019nCoV on hard, nonporous surfaces."" ly assumes that because the label for Lysol disinfectant spray includes mention of its effectiveness against """"coronavirus,"""" media descriptions of a current outbreak as being caused by a new virus are """"B.S.""",0
1880,Meth in Louisiana is contaminated with the Zika virus.,"On 29 December 2018, the Police Department of Harahan, Louisiana, posted a warning on Facebook advising local residents that if they had recently purchased meth in any area of Louisiana, they should bring it to their local police department to ensure it was not contaminated with the Zika virus: If you have recently purchased meth in any area of Louisiana it may be contaminated with the Zika Virus. In fact, no meth in Louisiana or anywhere else is believed to be contaminated with the Zika virus which is typically transmitted through mosquito bites. Indeed, the Harahan police chief admitted that the department never had reason to believe Zika was able to contaminate meth and that they have no way to test for Zika in meth, asserting that the Facebook post was a stunt to raise awareness of drug abuse: Harahan Police Chief Tim Walker acknowledged that the post was just a stunt to raise awareness of drug abuse and that its not actually possible to have Zika virus in methamphetamine.",0
700,There are places in America that are among the most dangerous in the world. You go to places like Oakland. Or Ferguson. The crime numbers are worse. Seriously.,"""""There are places in America that are among the most dangerous in the world. ""The most dangerous place in America is Donald Trump's mouth."" So where are the most d

In [17]:
train.to_csv('../data/train.csv', index=False)
test.to_csv('../data/test.csv', index=False)
val.to_csv('../data/val.csv', index=False)